In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import numpy as np
import lightgbm as lgb
import pandas as pd
from kaggle.competitions import twosigmanews
import matplotlib.pyplot as plt
import random
from datetime import datetime, date
from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
import time
pd.options.mode.chained_assignment = None
from datetime import timedelta
%matplotlib inline
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
cf.go_offline()

In [ ]:
# official way to get the data
from kaggle.competitions import twosigmanews
env = twosigmanews.make_env()
print('Done!')

In [ ]:
(market_train_df, news_train_df) = env.get_training_data()
print(market_train_df['time'].head)
print(market_train_df.shape)

In [ ]:
#import datetime as dt
#from time import strftime
#market_train_df['date'] = market_train_df['time'].dt.strftime('%Y-%m-%d')
#import pytz
#print(date(2010, 1, 1))

market_train_df['date'] = market_train_df['time'].dt.strftime('%Y-%m-%d')
#market_train_df['date'] = market_train_df['time'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
print(market_train_df['time'].dtype) 
print(market_train_df['date'].dtype) 
market_train_df['time'] = market_train_df['time'].dt.date
market_train_df = market_train_df.loc[market_train_df['time']>=date(2010, 1, 1)]
print(market_train_df.shape)

In [ ]:
#异常值修复
def sampleAssetData(assetCode, date, numDays):
    d = datetime.strptime(date,'%Y-%m-%d')
    start = d - timedelta(days=numDays)
    end = d + timedelta(days=numDays)
    return market_train_df[(market_train_df['assetCode'] == assetCode)
                             & (market_train_df['time'] >= start.date())
                             & (market_train_df['time'] <= end.date())].copy()

def updateRawReturns(assetData, indices):
    rowsToUpdate1 = assetData[(assetData.index >= indices[0][0]) & (assetData.index <= indices[0][1])]
    for index, row in rowsToUpdate1.iterrows():
        market_train_df.loc[[index],['returnsClosePrevRaw1']] = assetData['close'].pct_change()
        market_train_df.loc[[index],['returnsOpenPrevRaw1']] = assetData['open'].pct_change()
    rowsToUpdate2 = assetData[(assetData.index >= indices[1][0]) & (assetData.index <= indices[1][1])]
    for index, row in rowsToUpdate2.iterrows():
        market_train_df.loc[[index],['returnsClosePrevRaw10']] = assetData['close'].pct_change(periods=10)
        market_train_df.loc[[index],['returnsOpenPrevRaw10']] = assetData['open'].pct_change(periods=10)

def estimateMktresReturn(sampleData, mktresCol, index):
    sampleData['ones'] = 1
    sampleData.dropna(inplace=True)
    rawCol = mktresCol.replace('Mktres','Raw')
    A = sampleData[[rawCol,'ones']]
    y = sampleData[mktresCol]
    m, c = np.linalg.lstsq(A,y,rcond=-1)[0]
    return c + m * market_train_df.loc[index,rawCol]

def updateMktresReturns(assetCode, assetData, indices):
    # update range of values for returnsClosePrevMktres1 & returnsOpenPrevMktres1
    sample1 = assetData[(assetData.index < indices[2][0]) | (assetData.index > indices[2][1])]
    rowsToUpdate1 = assetData[(assetData.index >= indices[2][0]) & (assetData.index <= indices[2][1])]
    for index, row in rowsToUpdate1.iterrows():
        market_train_df.loc[[index],['returnsClosePrevMktres1']] = estimateMktresReturn(sample1,'returnsClosePrevMktres1',index)
        market_train_df.loc[[index],['returnsOpenPrevMktres1']] = estimateMktresReturn(sample1,'returnsOpenPrevMktres1',index)
    # update range of values for returnsClosePrevMktres10 & returnsOpenPrevMktres10
    sample2 = assetData[(assetData.index < indices[3][0]) | (assetData.index > indices[3][1])]
    rowsToUpdate2 = assetData[(assetData.index >= indices[3][0]) & (assetData.index <= indices[3][1])]
    l = []
    for index, row in rowsToUpdate2.iterrows():
        market_train_df.loc[[index],['returnsClosePrevMktres10']] = estimateMktresReturn(sample2,'returnsClosePrevMktres10',index)
        est = estimateMktresReturn(sample2,'returnsOpenPrevMktres10',index)
        l.append(est)
        market_train_df.loc[[index],['returnsOpenPrevMktres10']] = est
    # update range of values for returnsOpenNextMktres10
    rowsToUpdate3 = assetData[(assetData.index >= indices[4][0]) & (assetData.index <= indices[4][1])]
    i = 0
    for index, row in rowsToUpdate3.iterrows():
        market_train_df.loc[[index],['returnsOpenNextMktres10']] = l[i]
        i += 1
        
def fixBadReturnData(assetCode, badDate, badIndex, badReturnDataRanges, dayWindow):
    # store copy of bad data window
    badDataWindow = sampleAssetData(assetCode,badDate,dayWindow)
    badDataWindow.reset_index(inplace=True)
    # store indices needed to update raw and mktres return data
    newIdx = badDataWindow[badDataWindow['index'] == badIndex].index[0]
    indices = [
        # range of bad data for... returnsClosePrevRaw1 & returnsOpenPrevRaw1
        [badIndex,badDataWindow.loc[newIdx+badReturnDataRanges[0],'index']],
        # returnsClosePrevRaw10 & returnsOpenPrevRaw10
        [badIndex,badDataWindow.loc[newIdx+badReturnDataRanges[1],'index']],
        # returnsClosePrevMktres1 & returnsOpenPrevMktres1
        [badIndex,badDataWindow.loc[newIdx+badReturnDataRanges[2],'index']],
        # returnsClosePrevMktres10 & returnsOpenPrevMktres10
        [badIndex,badDataWindow.loc[newIdx+badReturnDataRanges[3],'index']],
        # returnsOpenNextMktres10
        [badDataWindow.loc[newIdx+badReturnDataRanges[4],'index'],badDataWindow.loc[newIdx+badReturnDataRanges[5],'index']]
    ]
    badDataWindow.set_index('index',inplace=True)
    # correct bad raw return data
    updateRawReturns(badDataWindow,indices)
    # estimate affected mktres return data
    updateMktresReturns(assetCode,badDataWindow,indices)
    

In [ ]:
# bad volume, open, and close for ZNGA.O on 2016-07-06
assetCode = 'ZNGA.O'
badDate = '2016-07-06'
badIndex = market_train_df[(market_train_df['assetCode'] == assetCode) & (market_train_df['date'] == badDate)].index[0]
# correct bad data
market_train_df.loc[[badIndex],['volume']] = 19213100
market_train_df.loc[[badIndex],['open']] = 2.64
market_train_df.loc[[badIndex],['close']] = 2.75
# ranges of affected return data
#   integers specify how many rows of data need to be updated for different return columns, in reference to "badDate" row
badReturnDataRanges = [1,10,20,20,-11,9]
# sample data window (on either side of "badDate")
sampleWindow = 45
# fix bad return data in market_train_df
fixBadReturnData(assetCode,badDate,badIndex,badReturnDataRanges,sampleWindow)

# bad volume, open, and close for FLEX.O on 2016-07-06
assetCode = 'FLEX.O'
badDate = '2016-07-06'
badIndex = market_train_df[(market_train_df['assetCode'] == assetCode) & (market_train_df['date'] == badDate)].index[0]
# correct bad data
market_train_df.loc[[badIndex],['volume']] = 5406600
market_train_df.loc[[badIndex],['open']] = 11.580
market_train_df.loc[[badIndex],['close']] = 11.750
# ranges of affected return data
badReturnDataRanges = [1,10,20,20,-11,9]
# sample data window (on either side of "badDate")
sampleWindow = 45
# fix bad return data in market_train_df
fixBadReturnData(assetCode,badDate,badIndex,badReturnDataRanges,sampleWindow)

# bad volume, open, and close for SHLD.O on 2016-07-06
assetCode = 'SHLD.O'
badDate = '2016-07-06'
badIndex = market_train_df[(market_train_df['assetCode'] == assetCode) & (market_train_df['date'] == badDate)].index[0]
# correct bad data
market_train_df.loc[[badIndex],['volume']] = 279300
market_train_df.loc[[badIndex],['open']] = 12.8900
market_train_df.loc[[badIndex],['close']] = 13.1400
# ranges of affected return data
badReturnDataRanges = [1,10,20,20,-11,9]
# sample data window (on either side of "badDate")
sampleWindow = 45
# fix bad return data in market_train_df
fixBadReturnData(assetCode,badDate,badIndex,badReturnDataRanges,sampleWindow)

# bad volume, open, and close for MAT.O on 2016-07-06
assetCode = 'MAT.O'
badDate = '2016-07-06'
badIndex = market_train_df[(market_train_df['assetCode'] == assetCode) & (market_train_df['date'] == badDate)].index[0]
# correct bad data
market_train_df.loc[[badIndex],['volume']] = 3242100
market_train_df.loc[[badIndex],['open']] = 32.13
market_train_df.loc[[badIndex],['close']] = 31.52
# ranges of affected return data
badReturnDataRanges = [1,10,20,20,-11,9]
# sample data window (on either side of "badDate")
sampleWindow = 45
# fix bad return data in market_train_df
fixBadReturnData(assetCode,badDate,badIndex,badReturnDataRanges,sampleWindow)

# bad volume, open, and close for BBBY.O on 2016-07-06
assetCode = 'BBBY.O'
badDate = '2016-07-06'
badIndex = market_train_df[(market_train_df['assetCode'] == assetCode) & (market_train_df['date'] == badDate)].index[0]
# correct bad data
market_train_df.loc[[badIndex],['volume']] = 4205500
market_train_df.loc[[badIndex],['open']] = 42.23
market_train_df.loc[[badIndex],['close']] = 43.55
# ranges of affected return data
badReturnDataRanges = [1,10,20,20,-11,9]
# sample data window (on either side of "badDate")
sampleWindow = 45
# fix bad return data in market_train_df
fixBadReturnData(assetCode,badDate,badIndex,badReturnDataRanges,sampleWindow)

# bad volume, open, and close for DISH.O on 2016-07-06
assetCode = 'DISH.O'
badDate = '2016-07-06'
badIndex = market_train_df[(market_train_df['assetCode'] == assetCode) & (market_train_df['date'] == badDate)].index[0]
# correct bad data
market_train_df.loc[[badIndex],['volume']] = 2303300
market_train_df.loc[[badIndex],['open']] = 50.06
market_train_df.loc[[badIndex],['close']] = 51.33
# ranges of affected return data
badReturnDataRanges = [1,10,20,20,-11,9]
# sample data window (on either side of "badDate")
sampleWindow = 45
# fix bad return data in market_train_df
fixBadReturnData(assetCode,badDate,badIndex,badReturnDataRanges,sampleWindow)

# bad volume, open, and close for NDAQ.O on 2016-07-06
assetCode = 'NDAQ.O'
badDate = '2016-07-06'
badIndex = market_train_df[(market_train_df['assetCode'] == assetCode) & (market_train_df['date'] == badDate)].index[0]
# correct bad data
market_train_df.loc[[badIndex],['volume']] = 733400
market_train_df.loc[[badIndex],['open']] = 64.64
market_train_df.loc[[badIndex],['close']] = 64.74
# ranges of affected return data
badReturnDataRanges = [1,10,20,20,-11,9]
# sample data window (on either side of "badDate")
sampleWindow = 45
# fix bad return data in market_train_df
fixBadReturnData(assetCode,badDate,badIndex,badReturnDataRanges,sampleWindow)

# bad volume, open, and close for PCAR.O on 2016-07-06
assetCode = 'PCAR.O'
badDate = '2016-07-06'
badIndex = market_train_df[(market_train_df['assetCode'] == assetCode) & (market_train_df['date'] == badDate)].index[0]
# correct bad data
market_train_df.loc[[badIndex],['volume']] = 2394300
market_train_df.loc[[badIndex],['open']] = 50.16
market_train_df.loc[[badIndex],['close']] = 50.79
# ranges of affected return data
badReturnDataRanges = [1,10,20,20,-11,9]
# sample data window (on either side of "badDate")
sampleWindow = 45
# fix bad return data in market_train_df
fixBadReturnData(assetCode,badDate,badIndex,badReturnDataRanges,sampleWindow)

# bad volume, open, and close for PZZA.O on 2016-07-06
assetCode = 'PZZA.O'
badDate = '2016-07-06'
badIndex = market_train_df[(market_train_df['assetCode'] == assetCode) & (market_train_df['date'] == badDate)].index[0]
# correct bad data
market_train_df.loc[[badIndex],['volume']] = 185100
market_train_df.loc[[badIndex],['open']] = 67.86
market_train_df.loc[[badIndex],['close']] = 67.91
# ranges of affected return data
badReturnDataRanges = [1,10,20,20,-11,9]
# sample data window (on either side of "badDate")
sampleWindow = 45
# fix bad return data in market_train_df
fixBadReturnData(assetCode,badDate,badIndex,badReturnDataRanges,sampleWindow)

In [ ]:
#排异
#market_train_df = market_train_df[market_train_df.assetName !='Unknown']
market_train_df = market_train_df[market_train_df.assetCode != 'PGN.N']
market_train_df = market_train_df[market_train_df.assetCode != 'EBRYY.OB']
print(market_train_df.shape)
#market_train_df = market_train_df[market_train_df.assetCode != 'TW.N']
print(market_train_df.shape)

#数据增强——缩放
print(market_train_df.columns.values.tolist())
print(market_train_df.shape)
market_enhance_df = market_train_df[['open','close','time', 'assetCode','assetName', 'universe','volume' ]].sample(frac=0.5).copy()
#print(market_enhance_df.dtype)
print(market_enhance_df['assetCode'].dtype)
f = lambda x: x+".E"
#df['browse_nodes'] = df['browse_nodes'].map(f)
market_enhance_df['assetCode'] = market_enhance_df['assetCode'].map(f)
print(market_enhance_df['assetCode'].head())
#market_enhance_df['assetCode'] = [ 'E. % s' % i for i in market_train_df['assetCode']]
#market_enhance_df['assetCode']+".E"
market_enhance_df['open'] = market_enhance_df['open'].map(lambda x: x/2)
#market_enhance_df['open']/2
market_enhance_df['close'] = market_enhance_df['open'].map(lambda x: x/2)
#market_enhance_df['close']/2
market_enhance_df['returnsClosePrevRaw1'] = market_enhance_df['close'].pct_change()
market_enhance_df['returnsOpenPrevRaw1'] = market_enhance_df['open'].pct_change()
market_enhance_df['returnsClosePrevRaw1'] = market_enhance_df['close'].pct_change(periods=10)
market_enhance_df['returnsOpenPrevRaw1'] = market_enhance_df['open'].pct_change(periods=10)
market_enhance_df['returnsClosePrevMktres1'] = 0
# market_enhance_df['returnsOpenPrevMktres1'].fillna(0)
# market_enhance_df['returnsClosePrevMktres10'].fillna(0)
# market_enhance_df['returnsOpenPrevMktres10'].fillna(0)
# market_enhance_df['returnsOpenNextMktres10'].fillna(0)
print(market_enhance_df.shape)
market_train_df = market_train_df.append(market_enhance_df)


# def estimateMktresReturn(sampleData, mktresCol, index):
#     sampleData['ones'] = 1
#     sampleData.dropna(inplace=True)
#     rawCol = mktresCol.replace('Mktres','Raw')
#     A = sampleData[[rawCol,'ones']]
#     y = sampleData[mktresCol]
#     m, c = np.linalg.lstsq(A,y,rcond=-1)[0]
#     return c + m * market_train_df.loc[index,rawCol]

# def updateMktresReturns(assetCode, assetData, indices):
#     # update range of values for returnsClosePrevMktres1 & returnsOpenPrevMktres1
#     sample1 = assetData[(assetData.index < indices[2][0]) | (assetData.index > indices[2][1])]
#     rowsToUpdate1 = assetData[(assetData.index >= indices[2][0]) & (assetData.index <= indices[2][1])]
#     for index, row in rowsToUpdate1.iterrows():
#         market_train_df.loc[[index],['returnsClosePrevMktres1']] = estimateMktresReturn(sample1,'returnsClosePrevMktres1',index)
#         market_train_df.loc[[index],['returnsOpenPrevMktres1']] = estimateMktresReturn(sample1,'returnsOpenPrevMktres1',index)
#     # update range of values for returnsClosePrevMktres10 & returnsOpenPrevMktres10
#     sample2 = assetData[(assetData.index < indices[3][0]) | (assetData.index > indices[3][1])]
#     rowsToUpdate2 = assetData[(assetData.index >= indices[3][0]) & (assetData.index <= indices[3][1])]
#     l = []
#     for index, row in rowsToUpdate2.iterrows():
#         market_train_df.loc[[index],['returnsClosePrevMktres10']] = estimateMktresReturn(sample2,'returnsClosePrevMktres10',index)
#         est = estimateMktresReturn(sample2,'returnsOpenPrevMktres10',index)
#         l.append(est)
#         market_train_df.loc[[index],['returnsOpenPrevMktres10']] = est
#     # update range of values for returnsOpenNextMktres10
#     rowsToUpdate3 = assetData[(assetData.index >= indices[4][0]) & (assetData.index <= indices[4][1])]
#     i = 0
#     for index, row in rowsToUpdate3.iterrows():
#         market_train_df.loc[[index],['returnsOpenNextMktres10']] = l[i]
#         i += 1
        

In [ ]:
print(market_train_df.shape)

#quant EDA
def generate_quant_df(market_train_df):
    #EDA
    market_train_df['price_diff'] = market_train_df['close'] - market_train_df['open'] 
    #grouped = market_train_df.groupby('time').agg({'price_diff': ['std', 'min']}).reset_index()
    #print(market_train_df.sort_values('price_diff')[:10])
    
    #Moving average
    market_train_df['MA_7MA'] = market_train_df['close'].rolling(window=7).mean()
    market_train_df['MA_15MA'] = market_train_df['close'].rolling(window=15).mean()
    market_train_df['MA_30MA'] = market_train_df['close'].rolling(window=30).mean()
    market_train_df['MA_60MA'] = market_train_df['close'].rolling(window=60).mean()

    #Exponential Moving Average
    ewma = pd.Series.ewm
    market_train_df['close_30EMA'] = ewma(market_train_df["close"], span=30).mean()

    #MACD
    market_train_df['close_26EMA'] = ewma(market_train_df["close"], span=26).mean()
    market_train_df['close_12EMA'] = ewma(market_train_df["close"], span=12).mean()
    market_train_df['MACD'] = market_train_df['close_12EMA'] - market_train_df['close_26EMA']
    #market_train_df.drop(columns=['close_26EMA', 'close_12EMA'])
    
    #Bollinger Band
    no_of_std = 2
    #market_train_df['MA_7MA'] = market_train_df['close'].rolling(window=7).mean()
    market_train_df['MA_7MA_std'] = market_train_df['close'].rolling(window=7).std() 
    market_train_df['MA_7MA_BB_high'] = market_train_df['MA_7MA'] + no_of_std * market_train_df['MA_7MA_std']
    market_train_df['MA_7MA_BB_low'] = market_train_df['MA_7MA'] - no_of_std * market_train_df['MA_7MA_std']
    #market_train_df.drop(columns=['MA_7MA_std'])
    
    market_train_df['MA_15MA_std'] = market_train_df['close'].rolling(window=15).std() 
    market_train_df['MA_15MA_BB_high'] = market_train_df['MA_15MA'] + no_of_std * market_train_df['MA_15MA_std']
    market_train_df['MA_15MA_BB_low'] = market_train_df['MA_15MA'] - no_of_std * market_train_df['MA_15MA_std']

    market_train_df['MA_30MA_std'] = market_train_df['close'].rolling(window=30).std() 
    market_train_df['MA_30MA_BB_high'] = market_train_df['MA_30MA'] + no_of_std * market_train_df['MA_30MA_std']
    market_train_df['MA_30MA_BB_low'] = market_train_df['MA_30MA'] - no_of_std * market_train_df['MA_30MA_std']


    #RSI
    def rsiFunc(prices, n=14):
        deltas = np.diff(prices)
        seed = deltas[:n+1]
        up = seed[seed>=0].sum()/n
        down = -seed[seed<0].sum()/n
        rs = up/down
        rsi = np.zeros_like(prices)
        rsi[:n] = 100. - 100./(1.+rs)

        for i in range(n, len(prices)):
            delta = deltas[i-1] # cause the diff is 1 shorter

            if delta>0:
                upval = delta
                downval = 0.
            else:
                upval = 0.
                downval = -delta

            up = (up*(n-1) + upval)/n
            down = (down*(n-1) + downval)/n

            rs = up/down
            rsi[i] = 100. - 100./(1.+rs)

        return rsi
    print(rsiFunc(market_train_df['close'].values, 6).dtype)
    market_train_df['rsi_6'] = rsiFunc(market_train_df['close'].values, 6)
    market_train_df['rsi_14'] = rsiFunc(market_train_df['close'].values, 14)
    market_train_df['rsi_20'] = rsiFunc(market_train_df['close'].values, 20)

    #Volume moving avreage
    market_train_df['VMA_7MA'] = market_train_df['volume'].rolling(window=7).mean()
    market_train_df['VMA_15MA'] = market_train_df['volume'].rolling(window=15).mean()
    market_train_df['VMA_30MA'] = market_train_df['volume'].rolling(window=30).mean()
    market_train_df['VMA_60MA'] = market_train_df['volume'].rolling(window=60).mean()
    
#generate_quant_df(market_train_df)

In [ ]:
from multiprocessing import Pool

def create_lag(df_code,n_lag=[3,7,14,],shift_size=1):
    code = df_code['assetCode'].unique()
    
    for col in return_features:
        for window in n_lag:
            rolled = df_code[col].shift(shift_size).rolling(window=window)
            lag_mean = rolled.mean()
            lag_max = rolled.max()
            lag_min = rolled.min()
            lag_std = rolled.std()
            df_code['%s_lag_%s_mean'%(col,window)] = lag_mean
            df_code['%s_lag_%s_max'%(col,window)] = lag_max
            df_code['%s_lag_%s_min'%(col,window)] = lag_min
#             df_code['%s_lag_%s_std'%(col,window)] = lag_std
    return df_code.fillna(-1)

def generate_lag_features(df,n_lag = [3,7,14]):
    features = ['time', 'assetCode', 'assetName', 'volume', 'close', 'open',
       'returnsClosePrevRaw1', 'returnsOpenPrevRaw1',
       'returnsClosePrevMktres1', 'returnsOpenPrevMktres1',
       'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',
       'returnsClosePrevMktres10', 'returnsOpenPrevMktres10',
       'returnsOpenNextMktres10', 'universe']
    
    assetCodes = df['assetCode'].unique()
    print(assetCodes)
    all_df = []
    df_codes = df.groupby('assetCode')
    df_codes = [df_code[1][['time','assetCode']+return_features] for df_code in df_codes]
    print('total %s df'%len(df_codes))
    
    pool = Pool(4)
    all_df = pool.map(create_lag, df_codes)
    
    new_df = pd.concat(all_df)  
    new_df.drop(return_features,axis=1,inplace=True)
    pool.close()
    
    return new_df

In [ ]:
# return_features = ['close']
# new_df = generate_lag_features(market_train_df,n_lag = 5)
# market_train_df = pd.merge(market_train_df,new_df,how='left',on=['time','assetCode'])

In [ ]:
return_features = ['returnsClosePrevMktres10','returnsClosePrevRaw10','open','close']
n_lag = [3,7,14]
new_df = generate_lag_features(market_train_df,n_lag=n_lag)
#generate_quant_df(market_train_df)
market_train_df = pd.merge(market_train_df,new_df,how='left',on=['time','assetCode'])

In [ ]:
print(market_train_df.columns)


In [ ]:
# return_features = ['open']
# new_df = generate_lag_features(market_train_df,n_lag=[3,7,14])
# market_train_df = pd.merge(market_train_df,new_df,how='left',on=['time','assetCode'])

In [ ]:
def mis_impute(data):
    for i in data.columns:
        if data[i].dtype == "object":
            data[i] = data[i].fillna("other")
        elif (data[i].dtype == "int64" or data[i].dtype == "float64"):
            data[i] = data[i].fillna(data[i].mean())
        else:
            pass
    return data

market_train_df = mis_impute(market_train_df)

In [ ]:
def data_prep(market_train):
    lbl = {k: v for v, k in enumerate(market_train['assetCode'].unique())}
    market_train['assetCodeT'] = market_train['assetCode'].map(lbl)
    market_train = market_train.dropna(axis=0)
    return market_train

market_train_df = data_prep(market_train_df)
# # check the shape
print(market_train_df.shape)

In [ ]:
from sklearn.preprocessing import LabelEncoder

up = market_train_df['returnsOpenNextMktres10'] >= 0


universe = market_train_df['universe'].values
d = market_train_df['time']

fcol = [c for c in market_train_df if c not in ['assetCode', 'assetCodes', 'assetCodesLen', 'assetName', 'audiences', 
                                             'firstCreated', 'headline', 'headlineTag', 'marketCommentary', 'provider', 
                                             'returnsOpenNextMktres10', 'sourceId', 'subjects', 'time', 'time_x', 'date' , 'universe','sourceTimestamp']]

X = market_train_df[fcol].values
up = up.values
r = market_train_df.returnsOpenNextMktres10.values

# Scaling of X values
# It is good to keep these scaling values for later
mins = np.min(X, axis=0)
maxs = np.max(X, axis=0)
rng = maxs - mins
X = 1 - ((maxs - X) / rng)

# Sanity check
assert X.shape[0] == up.shape[0] == r.shape[0]

from xgboost import XGBClassifier
from sklearn import model_selection
from sklearn.metrics import mean_squared_error
import time

X_train, X_test, up_train, up_test, r_train, r_test,u_train,u_test,d_train,d_test = model_selection.train_test_split(X, up, r,universe,d, test_size=0.25, random_state=99)


# te = market_train_df['time']>date(2015, 1, 1)

# tt = 0
# for tt,i in enumerate(te.values):
#     if i:
#         idx = tt
#         print(i,tt)
#         break
# print(idx)
# # for ind_tr, ind_te in tscv.split(X):
# #     print(ind_tr)
# X_train, X_test = X[:idx],X[idx:]

# up_train, up_test = up[:idx],up[idx:]
# r_train, r_test = r[:idx],r[idx:]
# u_train,u_test = universe[:idx],universe[idx:]
# d_train,d_test = d[:idx],d[idx:]

# train_data = lgb.Dataset(X_train, label=up_train.astype(int))
train_data = lgb.Dataset(X, label=up.astype(int))
test_data = lgb.Dataset(X_test, label=up_test.astype(int))

In [ ]:
# these are tuned params I found
x_1 = [0.19000424246380565, 2452, 212, 328, 202]
x_2 = [0.19016805202090095, 2583, 213, 312, 220]
print(up_train)
def exp_loss(p,y):
    y = y.get_label()
#     p = p.get_label()
    grad = -y*(1.0-1.0/(1.0+np.exp(-y*p)))
    hess = -(np.exp(y*p)*(y*p-1)-1)/((np.exp(y*p)+1)**2)
    
    return grad,hess

params_1 = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
#         'objective': 'regression',
        'learning_rate': x_1[0],
        'num_leaves': x_1[1],
        'min_data_in_leaf': x_1[2],
#         'num_iteration': x_1[3],
        'num_iteration': 239,
        'max_bin': x_1[4],
        'verbose': 1
    }

params_2 = {
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'binary',
#         'objective': 'regression',
        'learning_rate': x_2[0],
        'num_leaves': x_2[1],
        'min_data_in_leaf': x_2[2],
#         'num_iteration': x_2[3],
        'num_iteration': 172,
        'max_bin': x_2[4],
        'verbose': 1
    }

gbm_1 = lgb.train(params_1,
        train_data,
        num_boost_round=100,
        valid_sets=test_data,
        early_stopping_rounds=5,
#         fobj=exp_loss,
        )

gbm_2 = lgb.train(params_2,
        train_data,
        num_boost_round=100,
        valid_sets=test_data,
        early_stopping_rounds=5,
#         fobj=exp_loss,
        )



In [ ]:
confidence_test = (gbm_1.predict(X_test) + gbm_2.predict(X_test))/2
confidence_test = (confidence_test-confidence_test.min())/(confidence_test.max()-confidence_test.min())
confidence_test = confidence_test*2-1
print(max(confidence_test),min(confidence_test))

# calculation of actual metric that is used to calculate final score
r_test = r_test.clip(-1,1) # get rid of outliers. Where do they come from??
x_t_i = confidence_test * r_test * u_test
data = {'day' : d_test, 'x_t_i' : x_t_i}
df = pd.DataFrame(data)
x_t = df.groupby('day').sum().values.flatten()
mean = np.mean(x_t)
std = np.std(x_t)
score_test = mean / std
print(score_test)


In [ ]:
import gc
del X_train,X_test
gc.collect()

In [ ]:
#prediction
days = env.get_prediction_days()
n_days = 0
prep_time = 0
prediction_time = 0
packaging_time = 0
total_market_obs_df = []
for (market_obs_df, news_obs_df, predictions_template_df) in days:
    n_days +=1
    if (n_days%50==0):
        print(n_days,end=' ')
    t = time.time()
    market_obs_df['time'] = market_obs_df['time'].dt.date
    
    return_features = ['returnsClosePrevMktres10','returnsClosePrevRaw10','open','close']
    total_market_obs_df.append(market_obs_df)
    if len(total_market_obs_df)==1:
        history_df = total_market_obs_df[0]
    else:
        history_df = pd.concat(total_market_obs_df[-(np.max(n_lag)+1):])
    print(history_df)
    
    new_df = generate_lag_features(history_df,n_lag=[3,7,14])
   #generate_quant_df(market_obs_df)
    market_obs_df = pd.merge(market_obs_df,new_df,how='left',on=['time','assetCode'])
    
#     return_features = ['open']
#     new_df = generate_lag_features(market_obs_df,n_lag=[3,7,14])
#     market_obs_df = pd.merge(market_obs_df,new_df,how='left',on=['time','assetCode'])
    
    market_obs_df = mis_impute(market_obs_df)
    
    market_obs_df = data_prep(market_obs_df)
    
#     market_obs_df = market_obs_df[market_obs_df.assetCode.isin(predictions_template_df.assetCode)]
    
    X_live = market_obs_df[fcol].values
    X_live = 1 - ((maxs - X_live) / rng)
    prep_time += time.time() - t
    
    t = time.time()
    lp = (gbm_1.predict(X_live) + gbm_2.predict(X_live))/2
    prediction_time += time.time() -t
    
    t = time.time()
    
    confidence = lp
    confidence = (confidence-confidence.min())/(confidence.max()-confidence.min())
    confidence = confidence * 2 - 1
    
    preds = pd.DataFrame({'assetCode':market_obs_df['assetCode'],'confidence':confidence})
    predictions_template_df = predictions_template_df.merge(preds,how='left').drop('confidenceValue',axis=1).fillna(0).rename(columns={'confidence':'confidenceValue'})
    env.predict(predictions_template_df)
    packaging_time += time.time() - t
    
env.write_submission_file()
sub  = pd.read_csv("submission.csv")